# Imports

In [1]:
import pandas as pd
import numpy as np

# Question 3: Neural Networks

## Part A: Custom Implementation

Let's begin with the dataset given in the PDF:

In [7]:
feat1 = np.array([4, 4, 1, 2.5, 4.9, 1.9, 3.5, 0.5, 2, 4.5])
feat2 = np.array([2.9, 4, 2.5, 1, 4.5, 1.9, 4, 1.5, 2.1, 2.5])
labels = np.array([1, 1, -1, -1, 1, -1, 1, -1, -1, 1])
df = pd.DataFrame(data=dict({
    "x1": feat1,
    "x2": feat2,
    "y_true": labels 
}))
df.head(10)

,x1,x2,y_true
0,4.0,2.9,1
1,4.0,4.0,1
2,1.0,2.5,-1
3,2.5,1.0,-1
4,4.9,4.5,1
5,1.9,1.9,-1
6,3.5,4.0,1
7,0.5,1.5,-1
8,2.0,2.1,-1
9,4.5,2.5,1
